# BentoML Example:  Deploy to AWS Lambda

[BentoML](http://bentoml.ai) is an open source framework for building, shipping and running machine learning services. It provides high-level APIs for defining an ML service and packaging its artifacts, source code, dependencies, and configurations into a production-system-friendly format that is ready for deployment.

This notebook demonstrates how to use BentoML to deploy a machine learning model as a serverless REST API endpoint to AWS Lambda. For this demo, we are using the [Sentiment Analysis with Scikit-learn](https://github.com/bentoml/BentoML/blob/master/examples/sklearn-sentiment-clf/sklearn-sentiment-clf.ipynb) example, using dataset from [Sentiment140](http://help.sentiment140.com/for-students/).

![Impression](https://www.google-analytics.com/collect?v=1&tid=UA-112879361-3&cid=555&t=event&ec=nb&ea=open&el=official-example&dt=deploy-with-serverless)

In [ ]:
%reload_ext autoreload
%autoreload 2
%matplotlib inline

In [ ]:
!pip install bentoml
!pip install sklearn pandas numpy

In [ ]:
import bentoml
import numpy as np
import pandas as pd
from sklearn.feature_extraction.text import CountVectorizer
from sklearn.linear_model import LogisticRegression
from sklearn.metrics import classification_report, roc_auc_score, roc_curve
from sklearn.pipeline import Pipeline

# Prepare Dataset

In [ ]:
%%bash

if [ ! -f ./trainingandtestdata.zip ]; then
    wget -q http://cs.stanford.edu/people/alecmgo/trainingandtestdata.zip
    unzip -n trainingandtestdata.zip
fi

In [ ]:
columns = ['polarity', 'tweetid', 'date', 'query_name', 'user', 'text']
dftrain = pd.read_csv('training.1600000.processed.noemoticon.csv',
                      header = None,
                      encoding ='ISO-8859-1')
dftest = pd.read_csv('testdata.manual.2009.06.14.csv',
                     header = None,
                     encoding ='ISO-8859-1')
dftrain.columns = columns
dftest.columns = columns

# Model Training

In [ ]:
sentiment_lr = Pipeline([
                         ('count_vect', CountVectorizer(min_df = 100,
                                                        ngram_range = (1,1),
                                                        stop_words = 'english')), 
                         ('lr', LogisticRegression())])
sentiment_lr.fit(dftrain.text, dftrain.polarity)

In [ ]:
Xtest, ytest = dftest.text[dftest.polarity!=2], dftest.polarity[dftest.polarity!=2]
print(classification_report(ytest,sentiment_lr.predict(Xtest)))

In [ ]:
sentiment_lr.predict([Xtest[0]])

## Create BentoService for model serving

To package this trained model for model serving in production, you will need to create a new BentoML Service by subclassing it:

In [ ]:
%%writefile sentiment_lr_model.py

import pandas as pd
import bentoml
from bentoml.artifact import PickleArtifact
from bentoml.handlers import DataframeHandler

@bentoml.artifacts([PickleArtifact('model')])
@bentoml.env(pip_dependencies=['sklearn', 'numpy', 'pandas'])
class SentimentLRModel(bentoml.BentoService):
    
    @bentoml.api(DataframeHandler, typ='series')
    def predict(self, series):
        """
        predict expects pandas.Series as input
        """        
        return self.artifacts.model.predict(series)

## Save BentoService to file archive

In [ ]:
# 1) import the custom BentoService defined above
from sentiment_lr_model import SentimentLRModel

# 2) `pack` it with required artifacts
bento_service = SentimentLRModel.pack(model=sentiment_lr)

# 3) save BentoSerivce to file archive
saved_path = bento_service.save()

## Load saved BentoService

In [ ]:
# Load exported bentoML model archive from path
loaded_bento_service = bentoml.load(saved_path)

# Call predict on the restored BentoService
loaded_bento_service.predict(pd.Series(["hello", "hi"]))

# Deploy to AWS lambda

```bash
bentoml deployment create DEPLOYMENT_NAME --bento BENTO_TAG --platform PLATFORM
```
### Arguments:
* deployment name: The file path or s3 that contains BentoML bundles.

### Options:
* bento: REQUIRED. The bento model and version in format of name:version.
* platform: REQUIRED.  The platform that you want to deploy bentoml bundle to.  For serverless, we support aws-lambda, aws-lambda-py2 and gcp-function.
* region: OPTIONAL The cloud provider's region you want to deploy in.


In [ ]:
bento_tag = '{name}:{version}'.format(name=bento_service.name, version=bento_service.version)
print(bento_tag)

In [ ]:
!bentoml deployment create sentiment-serverless --bento {bento_tag} --platform aws-lambda --region us-west-2

### Make curl request to the Lambda endpoint

Replace the [URL] from the following command with the endpoints value from the previous output and then run it in a terminal
```bash
curl -i \
--header "Content-Type: application/json" \
--data '["good movie", "bad food"]' \
--request POST \
[URL]
```

Here is an example of running this command:

```
$curl -i \
> --header "Content-Type: application/json" \
> --data '["good movie", "bad food"]' \
> --request POST \
> https://yw38wha2tl.execute-api.ap-northeast-1.amazonaws.com/default/predict

HTTP/1.1 200 OK
Content-Type: application/json
Content-Length: 6
Connection: keep-alive
Date: Mon, 18 Nov 2019 13:10:12 GMT
x-amzn-RequestId: 245ac5ac-64cd-40b5-9ac4-8414eb1e5162
x-amz-apigw-id: DWy1xEPVtjMFrqQ=
X-Amzn-Trace-Id: Root=1-5dd29824-8d6d9748fc441bd0abc699e8;Sampled=0
X-Cache: Miss from cloudfront
Via: 1.1 4ac3d01dc034ade34c90e81091421c76.cloudfront.net (CloudFront)
X-Amz-Cf-Pop: SIN5-C1
X-Amz-Cf-Id: ZpFAY2QOzO5MZHs1cA5G2CmAy70x-s9Ch9ShoY22jK0w-gmWsLUpWQ==

[4, 0]
```

## Check deployment status

```
bentoml deployment describe DEPLOYMENT_NAME --namespace=NAMESPACE 
```

### Arguments:
* deployment_name

### Options:
* namespace: OPTIONAL. Name space of the deployment


In [ ]:
!bentoml deployment describe sentiment-serverless

## Delete serverless Deployment


```bash
bentoml deployment delete DEPLOYMENT_NAME --namespace NAMESPACE
```
### Arguments:
* deployment name
### Options:
* namespace: OPTIONAL. Name space of the deployment
* force: OPTIONAL. force deleting the deployment record and ignore errors when deleting cloud resources


In [ ]:
!bentoml deployment delete sentiment-serverless